authors: Julianna Cisewska, Clara Fayyad, Jagoda Hanuszewicz

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import brown
from nltk.probability import FreqDist
from collections import defaultdict

## Bonus question: PMI

In statistical NLP, we frequently make independence assumptions about relevant events which are not actually correct in reality. We are asking you to test the independence assumptions of unigram language models. Pointwise mutual information (PMI), is a measure of statistical dependence of the events 
Xt = w1 and Xt+1 = w2; 
C(w) is the absolute frequency and N is the size of the corpus. 
If the probability of the next word in the corpus being w2 is unaffected by the probability of the previous word being w1, then pmi(w1, w2) = 0; otherwise the PMI is positive or negative.

### Step 1. Calculate the PMI for all successive pairs (w1, w2) of words in the Brown corpus. 
Task: *Words (not word pairs!) that occur in the corpus less than 10 times should be ignored. List the 20 word pairs with the highest PMI value and the 20 word pairs with the lowest PMI value.*  
Our answer: We use brown corpus for this assignment. To calculate the PMI scores of the brown corpus we followed this tutorial: https://www.listendata.com/2022/06/pointwise-mutual-information-pmi.html

In [5]:
#downloads (do only once :))
nltk.download('brown')

[nltk_data] Downloading package brown to /Users/jago/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [13]:
#step 1 Tokenisation
tokens = brown.words()
words = [word.lower() for word in tokens if word.isalpha()]

#step 2 Filter out words with less than 10 occurences
frequent_words = {word for word, freq in freq_dist.items() if freq >= 10}

#step 3 Co-occurence matrix
def coocur_matrix(words):
    window_size = 2
    cooc_matrix = defaultdict(lambda: defaultdict(int))
    
    for i in range(len(words)):
        word_i = words[i]
        if word_i not in frequent_words:
            continue
        for j in range(max(0, i - window_size), min(len(words), i + window_size + 1)):
            if i == j:
                continue
            word_j = words[j]
            if word_j in frequent_words:
                cooc_matrix[word_i][word_j] += 1

    #step 3.5 Convert matrix to DataFrame
    df = pd.DataFrame.from_dict(cooc_matrix, orient='index').fillna(0).astype(int)
    return df

#step 4 Compute PMI score
def pmi(word1, word2, df):
    total_coocs = df.values.sum()

    #get frequencies, if word not present = 0
    cooc = df.at[word1, word2] if word1 in df.index and word2 in df.columns else 0
    freq_word1 = df.loc[word1].sum() if word1 in df.index else 0
    freq_word2 = df[word2].sum() if word2 in df.columns else 0

    #to avoid division by zero
    if cooc == 0 or freq_word1 == 0 or freq_word2 == 0:
        return 0

    #count probabilities
    p_xy = cooc / total_coocs
    p_x = freq_word1 / total_coocs
    p_y = freq_word2 / total_coocs

    return np.log2(p_xy / (p_x * p_y))

In [16]:
#check if the functions work well
df = coocur_matrix(words)
print("PMI('government', 'tax'):", pmi('government', 'tax', df))

PMI('government', 'tax'): 1.4686155845745006


In [17]:
#function that computes all PMI scores for our df
#WARNING: this cell takes a lot of time (uses coocurence matrix of >8000x8000 entries)
def compute_all_pmi(df):
    total_coocs = df.values.sum()
    pmi_scores = []

    for word1 in df.index:
        for word2 in df.columns:
            cooc = df.at[word1, word2]
            if cooc == 0:
                continue  # skip zero co-occurrence

            freq_word1 = df.loc[word1].sum()
            freq_word2 = df[word2].sum()

            p_xy = cooc / total_coocs
            p_x = freq_word1 / total_coocs
            p_y = freq_word2 / total_coocs

            pmi_val = np.log2(p_xy / (p_x * p_y))
            pmi_scores.append(((word1, word2), pmi_val))

    return pmi_scores

#compute and sort the scores
pmi_scores = compute_all_pmi(df)
pmi_sorted = sorted(pmi_scores, key=lambda x: x[1], reverse=True)  # highest first

In [18]:
#top 20 word pairs with the highest PMI value
print("\nTop 20 word pairs with highest PMI:")
for pair, score in pmi_sorted[:20]:
    print(f"{pair}: {score:.4f}")


Top 20 word pairs with highest PMI:
('tents', 'tents'): 15.2477
('hong', 'kong'): 14.3740
('kong', 'hong'): 14.3740
('cellulose', 'cellulose'): 13.8781
('lao', 'pathet'): 13.8375
('pathet', 'lao'): 13.8375
('viet', 'nam'): 13.7366
('nam', 'viet'): 13.7366
('simms', 'purdew'): 13.7158
('purdew', 'simms'): 13.7158
('wtv', 'antigen'): 13.7129
('antigen', 'wtv'): 13.7129
('el', 'paso'): 13.6271
('paso', 'el'): 13.6271
('tribune', 'herald'): 13.5217
('herald', 'tribune'): 13.5217
('lo', 'shu'): 13.2723
('shu', 'lo'): 13.2723
('non', 'non'): 13.2619
('puerto', 'rico'): 13.2043


In [19]:
#the 20 word pairs with the lowest PMI value
print("\nBottom 20 word pairs with lowest non-zero PMI:")
for pair, score in pmi_sorted[-20:]:
    print(f"{pair}: {score:.4f}")


Bottom 20 word pairs with lowest non-zero PMI:
('can', 'had'): -5.2432
('should', 'is'): -5.2487
('is', 'should'): -5.2487
('be', 'be'): -5.4462
('of', 'turned'): -5.5623
('turned', 'of'): -5.5623
('been', 'is'): -5.7016
('is', 'been'): -5.7016
('was', 'are'): -5.8488
('are', 'was'): -5.8488
('is', 'were'): -6.0521
('were', 'is'): -6.0521
('been', 'be'): -6.0648
('be', 'been'): -6.0648
('is', 'could'): -6.0907
('could', 'is'): -6.0907
('had', 'are'): -6.5085
('are', 'had'): -6.5085
('had', 'is'): -6.7364
('is', 'had'): -6.7364


Note for the negative scores from the tutorial:
```Negative PMI means words are co-occurring less than we expect by chance.```

### Step 2. Discuss the validity of the independence assumption for unigram models. 
Task: *Give 2-3 examples from your results to support your ideas.*  
**For the solution, see the report.**

### Step 3. Extend step 1 by researching and implementing both PMI and positive pointwise mutual information (PPMI). 
Task: *Do so on the entire Brown corpus and brown100. Document and submit your code with observations as comments in the same file as step 1.*  
Our answer: For understanding the difference between PPMI and PMI we followed this source https://towardsmachinelearning.org/positive-point-wise-mutual-information-ppmi/ and the same tutorial from the Step 1.

### Step 4. How does PPMI differ from PMI?
Task: *Both generally and given specific examples from your results?*  
**For the solution, see the report.**